In [1]:
%pip install --upgrade --force-reinstall z3-solver

  Using cached z3_solver-4.15.1.0-py3-none-macosx_13_0_arm64.whl.metadata (778 bytes)
Using cached z3_solver-4.15.1.0-py3-none-macosx_13_0_arm64.whl (37.5 MB)
  Attempting uninstall: z3-solver
    Found existing installation: z3-solver 4.15.1.0
    Uninstalling z3-solver-4.15.1.0:
      Successfully uninstalled z3-solver-4.15.1.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
from itertools import combinations
from z3 import *
import math
from utils import print_weekly_schedule
import time

## At least/most/exactly one encodings

### Naive pairwise

In [3]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name=''):
    return [Not(And(bool_vars[i], bool_vars[j])) for i in range(len(bool_vars) - 1) for j in range(i+1, len(bool_vars))]

def exactly_one_np(bool_vars, name=''):
    return at_most_one_np(bool_vars) + [at_least_one_np(bool_vars)] 



### Sequential  

In [4]:
def at_least_one_seq(bool_vars): # Identico a at_least_one_np
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name=''):
    constraints = []
    n = len(bool_vars)
    if n == 0: return BoolVal(True)
    if n == 1: return BoolVal(True) # Se c'è solo una variabile, al più una è sempre vera

    # Variabili ausiliarie 's' (i summatore sequenziali)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]

    # Clausole di bordo (primo e ultimo elemento)
    constraints.append(Or(Not(bool_vars[0]), s[0])) # x_0 -> s_0
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2]))) # x_{n-1} -> Not(s_{n-2})

    # Clausole per gli elementi intermedi
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))     # x_i -> s_i
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1]))) # x_i -> Not(s_{i-1})
        constraints.append(Or(Not(s[i-1]), s[i]))           # s_{i-1} -> s_i (monotonicità)
    
    return And(constraints)

def exactly_one_seq(bool_vars, name=''):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))


### Bitwise

In [5]:
def toBinary(num, length = None):
    num_bin = bin(num).split("b")[-1]
    if length:
        return "0"*(length - len(num_bin)) + num_bin
    return num_bin
    
def at_least_one_bw(bool_vars): # Identico a at_least_one_np
    return at_least_one_np(bool_vars)

def at_most_one_bw(bool_vars, name=''):
    constraints = []
    n = len(bool_vars)
    if n == 0: return BoolVal(True) # Caso base: nessun variabile, quindi al più uno è sempre vero
    
    m = math.ceil(math.log2(n)) # Numero di bit per la codifica binaria
    r = [Bool(f"r_{name}_{i}") for i in range(m)] # Variabili ausiliarie 'r' (i bit)
    binaries = [toBinary(idx, m) for idx in range(n)] # Codifiche binarie degli indici
    
    # *** CORREZIONE CRITICA QUI ***
    # Il loop originale era problematico. Ogni bool_vars[i] => codifica binaria di i
    for i in range(n):
        phi_parts = []
        for j in range(m):
            # Se il j-esimo bit della codifica binaria di 'i' è '1', allora r[j] deve essere True
            # altrimenti, Not(r[j]) deve essere True
            if binaries[i][j] == "1":
                phi_parts.append(r[j])
            else:
                phi_parts.append(Not(r[j]))
        # La clausola è: Not(bool_vars[i]) OR (congiunzione di tutti i phi_parts)
        # Questo è equivalente a: bool_vars[i] IMPLIES (congiunzione di tutti i phi_parts)
        constraints.append(Or(Not(bool_vars[i]), And(phi_parts))) # And(phi_parts) è la correzione

    return And(constraints) # Coniunzione di tutte le clausole

def exactly_one_bw(bool_vars, name=''):
    return And(at_least_one_bw(bool_vars), at_most_one_bw(bool_vars, name))

### Heule

In [6]:
# 4. Hierarchical Encoding (HE)
def at_least_one_he(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_he(bool_vars, name): # Il parametro 'name' è necessario e viene usato correttamente
    n = len(bool_vars)
    if n == 0: return BoolVal(True) # Aggiunto caso per lista vuota
    if n <= 4: # Caso base della ricorsione: usa l'encoding pairwise per liste piccole
        # CORREZIONE: Usa * per disimballare la lista di clausole restituita da at_most_one_np
        return And(*at_most_one_np(bool_vars))
    
    y = Bool(f"y_{name}") # Variabile ausiliaria con nome unico
    
    # c1_list sarà una LISTA di BoolRefs
    c1_list = at_most_one_np(bool_vars[:3] + [y]) 
    
    # c2_boolref sarà un singolo BoolRef dalla chiamata ricorsiva
    c2_boolref = at_most_one_he(bool_vars[3:] + [Not(y)], name + "_") 

    # CORREZIONE: Disimballa c1_list con * e passa c2_boolref separatamente
    # In questo modo, And riceve tutti gli argomenti come BoolRef separati.
    return And(*c1_list, c2_boolref)

def exactly_one_he(bool_vars, name):
    # exactly_one_he deve essere modificata in base alla correzione di at_most_one_he
    # at_most_one_he ora restituisce un singolo BoolRef quando n > 4.
    # at_least_one_he restituisce un singolo BoolRef (Or(...))
    return And(at_most_one_he(bool_vars, name), at_least_one_he(bool_vars))

# At least/most/exactly k encodings

### Naive

In [6]:
def at_least_k_np(bool_vars, k, name = ""):
    return at_most_k_np([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_np(bool_vars, k, name = ""):
    return And([Or([Not(x) for x in X]) for X in combinations(bool_vars, k + 1)])

def exactly_k_np(bool_vars, k, name = ""):
    return And(at_most_k_np(bool_vars, k, name), at_least_k_np(bool_vars, k, name))

# Final Solution

In [29]:
def at_most_k_seq(bool_vars, k, name=''):
    constraints = []
    n = len(bool_vars)
    
    if n == 0: return BoolVal(True)
    if k == 0: return And([Not(v) for v in bool_vars])
    if k >= n: return BoolVal(True)

    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n)] 
    
    # Inizializzazione per il primo elemento (bool_vars[0])
    constraints.append(Or(Not(bool_vars[0]), s[0][0])) 
    for j in range(1, k):
        constraints.append(Not(s[0][j]))

    # Propagazione per gli elementi successivi (i da 1 a n-1)
    for i in range(1, n):
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))

        for j in range(1, k):
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
        
        # Vincolo "Al più k" finale
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))

    return And(constraints)

def at_least_k_seq(bool_vars, k, name=''):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name + "_neg")

def exactly_k_seq(bool_vars, k, name=''):
    return And(at_most_k_seq(bool_vars, k, name), at_least_k_seq(bool_vars, k, name))


Non funziona per n=14

In [35]:
from itertools import combinations
from z3 import *
import math
from utils import print_weekly_schedule
import time


# --- 6. General K-Encoding (SEQ - Sequential Counter Encoding) ---

# --- Funzione principale per la risoluzione del problema STS ---
def sts_new_mine(n, exactly_one, at_most_k, timeout=None):
    NUM_TEAMS = n
    NUM_WEEKS = NUM_TEAMS - 1
    NUM_PERIODS_PER_WEEK = n // 2
    
    if NUM_TEAMS % 2 != 0:
        print(f"Errore: Il numero di squadre (n={NUM_TEAMS}) deve essere un numero pari.")
        return None, None, None

    s = Solver()
    if timeout:
        s.set("timeout", timeout)
        print(f"  Timeout del solver impostato a {timeout} ms.")
    
    games_vars = {}
    for i in range(NUM_TEAMS):
        for j in range(NUM_TEAMS):
            if i != j:
                for w in range(NUM_WEEKS):
                    for p in range(NUM_PERIODS_PER_WEEK):
                        games_vars[(i, j, w, p)] = Bool(f'g_{i+1}_{j+1}_{w+1}_{p+1}')
                        
    # --- Vincoli ---
    # 1. Ogni coppia di squadre gioca esattamente una volta (casa o trasferta)
    for i in range(NUM_TEAMS):
        for j in range(i+1, NUM_TEAMS):
            i_j_all_possible_matches = []
            for w in range(NUM_WEEKS):
                for p in range(NUM_PERIODS_PER_WEEK):
                    i_j_all_possible_matches.append(games_vars[(i, j, w, p)])
                    i_j_all_possible_matches.append(games_vars[(j, i, w, p)])
            s.add(exactly_one(i_j_all_possible_matches, f"pair_once_{i}_{j}"))
    
            
    # 2. Ogni squadra gioca esattamente una volta a settimana
    for i in range(NUM_TEAMS):
        for w in range(NUM_WEEKS):
            once_week_game = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for p in range(NUM_PERIODS_PER_WEEK):
                    once_week_game.append(games_vars[(i,j,w,p)])
                    once_week_game.append(games_vars[(j,i,w,p)])
            s.add(exactly_one(once_week_game, f"team_once_week_{i}_{w}"))
            
    # 3. Ogni squadra gioca al massimo due volte nello stesso periodo (es. 'periodo 1' di diverse settimane)
    for i in range(NUM_TEAMS):
        for p in range(NUM_PERIODS_PER_WEEK):
            num_games_period = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for w in range(NUM_WEEKS):
                    num_games_period.append(games_vars[(i,j,w,p)])
                    num_games_period.append(games_vars[(j,i,w,p)])
            s.add(at_most_k(num_games_period, 2, f"team_at_most_2_period_{i}_{p}"))
            
    # 4. In ogni periodo di ogni settimana si gioca esattamente una partita
    for w in range(NUM_WEEKS):
        for p in range(NUM_PERIODS_PER_WEEK):
            game_in_period = []
            for i in range(NUM_TEAMS):
                for j in range(i+1, NUM_TEAMS):
                    game_in_period.append(games_vars[(i,j,w,p)])
                    game_in_period.append(games_vars[(j,i,w,p)])
            s.add(exactly_one(game_in_period, f"slot_one_game_{w}_{p}"))
            
            
    # Symmetry Breaking (facoltativo, può aiutare il solver)
    if NUM_TEAMS >= 2 and NUM_WEEKS >= 1 and NUM_PERIODS_PER_WEEK >= 1:
        s.add(games_vars[(0, 1, 0, 0)])
    
    start_solve_time = time.time()
    check_result = s.check()
    elapsed_solve_time = time.time() - start_solve_time
    
    print(f"  Solver ha restituito: {check_result} dopo {elapsed_solve_time:.2f} secondi.")

    if check_result == sat:
        print("Solver found a SAT model!")
        m = s.model()
        solution_schedule = []
        for (i, j, w, p), var in games_vars.items():
            if m.evaluate(var):
                solution_schedule.append((i+1, j+1, w+1, p+1))
        print(f"Number of active matches in solution: {len(solution_schedule)}")
        return solution_schedule, games_vars, m
    else:
        print("No solution found.")
        return None, None, None


# --- Blocco principale per l'esecuzione del test ---
if __name__ == '__main__':
    # --- SCEGLI QUI GLI ENCODING DA USARE ---
    # Assegna le funzioni desiderate a queste variabili globali
    
    exactly_one = exactly_one_he 
    at_most_k = at_most_k_seq   

    n_test = 10
    test_timeout_ms = 5 * 60 * 1000 

    print(f"Sto usando exactly_one: {exactly_one.__name__}, e at_most_k: {at_most_k.__name__}")
    print(f"Tentativo di risolvere STS per {n_test} squadre...")
    
    match_list, match_dict, model = sts_new_mine(n_test, exactly_one, at_most_k, timeout=test_timeout_ms)

    if match_list:
        print_weekly_schedule(match_list, n_test)

Sto usando exactly_one: exactly_one_he, e at_most_k: at_most_k_seq
Tentativo di risolvere STS per 10 squadre...
  Timeout del solver impostato a 300000 ms.
  Solver ha restituito: sat dopo 3.73 secondi.
Solver found a SAT model!
Number of active matches in solution: 45

--- SCHEDULE DEL TORNEO ---
Numero di squadre: 10
Numero di settimane: 9
Periodi per settimana: 5
---------------------------

Settimana 1:
  Periodo 1: Squadra 1 (Casa) vs Squadra 2 (Trasferta)
  Periodo 2: Squadra 4 (Casa) vs Squadra 8 (Trasferta)
  Periodo 3: Squadra 10 (Casa) vs Squadra 7 (Trasferta)
  Periodo 4: Squadra 9 (Casa) vs Squadra 3 (Trasferta)
  Periodo 5: Squadra 5 (Casa) vs Squadra 6 (Trasferta)

Settimana 2:
  Periodo 1: Squadra 7 (Casa) vs Squadra 4 (Trasferta)
  Periodo 2: Squadra 5 (Casa) vs Squadra 10 (Trasferta)
  Periodo 3: Squadra 6 (Casa) vs Squadra 1 (Trasferta)
  Periodo 4: Squadra 2 (Casa) vs Squadra 3 (Trasferta)
  Periodo 5: Squadra 9 (Casa) vs Squadra 8 (Trasferta)

Settimana 3:
  Periodo